In [1]:
### Step-lenght
### sum of angles
### Number of spikes

In [1]:
from src.utile import *
from src.metrics import *
from src.visualisation import *
from src.activity_plotting import *

import numpy as np
import matplotlib.pyplot as plt
import time
import math
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [32]:
cameras = get_camera_names()
days = get_days_in_order()
camera, day = cameras[0], days[0]
print(camera, day)
batches = csv_of_the_day(camera, day, drop_out_of_scope=True)
BLOCK, DIR_CSV_LOCAL

23520289 20210910_1550


('block1', '../block1')

In [4]:
bdf = csv_of_the_day('23484201', days[25], is_back=False)